In [1]:
from models import MLP
from dataloader import read_bci_data
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
import torch.optim as optim

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
def seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed(500)

In [4]:
train_data, train_label, test_data, test_label = read_bci_data()

train_data = torch.Tensor(train_data)
train_label = torch.Tensor(train_label)

test_data = torch.Tensor(test_data)
test_label = torch.Tensor(test_label)

train = TensorDataset(train_data, train_label)
train_dataload = DataLoader(train,batch_size=64, shuffle=True)

test = TensorDataset(test_data, test_label)
test_dataload = DataLoader(test,batch_size=64, shuffle=False)

## MLP Training

In [8]:
## MLP Training
model=MLP()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)
loss_function = torch.nn.CrossEntropyLoss()

for epoch in range(300):
    
    train_loss = 0.0
    correct = 0
    
    for i, (data, label) in enumerate(train_dataload):
        label = label.type(torch.LongTensor)
        data = data.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        data = data.view(-1, 1500)
        output = model(data)
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        predicted = torch.max(output.data, 1)[1]
        correct += (predicted == label).sum().item()
        train_loss += loss.item()
    
    if((epoch+1)%100==0):
        print("-------{}th epoch-------".format(epoch+1))
        print("MLP Accuracy: ")
        print(correct/1080)
        print("Training Loss: ")
        print(train_loss)
        

-------100th epoch-------
MLP Accuracy: 
1.0
Training Loss: 
0.006566579482750967
-------200th epoch-------
MLP Accuracy: 
1.0
Training Loss: 
0.0056373173429165035
-------300th epoch-------
MLP Accuracy: 
1.0
Training Loss: 
0.004801952658453956


## MLP Testing

In [9]:
## MLP Testing
model.eval()
with torch.no_grad():
    
    correct = 0
    
    for i, (data, label) in enumerate(test_dataload):
        
        label = label.type(torch.LongTensor)
        data = data.to(device)
        label = label.to(device)
        
        data = data.view(-1, 1500)
        output = model(data)
        
        predicted = torch.max(output.data, 1)[1]
        correct += (predicted == label).sum().item()
    
    acc = correct/1080
    print("MLP Accuracy: ")
    print(acc)

MLP Accuracy: 
0.6703703703703704
